In [1]:
# Importing libraries

from ultralytics import YOLO
import cv2
import cvzone
import math
import numpy as np
from sort.sort import Sort

In [2]:
# Capture the video
cap = cv2.VideoCapture("D:\Videos\cars_1.mp4")

In [3]:
## Create a model of YOLOv8
model = YOLO("yolov8n.pt")

In [5]:
classNames = ['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 
              'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 
              'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 
              'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 
              'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 
              'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 
              'potted plant', 'bed', 'dining table', 'toilet', 'tv monitor', 'laptop', 'mouse', 'remote', 'keyboard', 
              'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 
              'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [4]:
## Create an instance for sorting (tracking the id)
tracker = Sort(max_age = 20, min_hits = 3, iou_threshold = 0.3)
          #max_age : Maximum number of frames that an object can be tracked without receiving any detections before the 
                     #object is considered lost and the tracker is removed.
                  #this parameter set to track objects for up to 20 frames without detection
          #min_hits : require at least 3 detections before considering an object a valid track
          #iou_threshold : Intersection over Union (IoU) threshold for matching detections to existing tracks.

In [6]:
## Values of limits (using these limits will create a line for the count of detection)
limits = [3, 280, 400, 280]   #starting & ending coordinates
         #[left width, left height, right width, right height]
    
totalCount = []

In [7]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (640, 360))

    # To import the graphics image of a car
    imgGraphics = cv2.imread("D:\Images\car_design.png", cv2.IMREAD_UNCHANGED)
    # Overlay the masking img on the main image
    frame = cvzone.overlayPNG(frame, imgGraphics, (0, 0))

    results = model(frame, stream=True, verbose=False, conf=0.5)
    # Create an array of detections
    detections = np.empty((0, 5))       #(0 rows & 5 cols)  |   #(dimension, values)
        # The array is intended to store detections, which likely consist of 5 values per detection

    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Create a Bounding Box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            # Confidence value
            conf = math.ceil((box.conf[0] * 100)) / 100
            # Class Name
            cls = int(box.cls[0])
            currentClass = classNames[cls]
            
            # Detect only car whose confidence is above 0.3
            if currentClass == 'car' and conf >= 0.3 :
                # Save the specific classes to the detection array / list
                currentArray = np.array([x1, y1, x2, y2, conf])
                        # [bounding box coordinates, confidence score]
                # To stack vertically (means to append each detections)  
                detections = np.vstack((detections, currentArray))

    # Update tracker with the list of detections
    resultsTracker = tracker.update(detections)
    # Create a line to count the no. of detections (whenever the Id crosses the line, 
        # so it will detect it as a count & will count it)
    cv2.line(frame, (limits[0], limits[1]), (limits[2], limits[3]), (0, 0, 255), 2)
             #limits[0], limits[1] : the starting point of the line
             #limits[2], limits[3] : the ending point of the line

    # To check Tracker Id's of each detected object (whether Id is changing or not)
    for res in resultsTracker:
        x1, y1, x2, y2, Id = res
        x1, y1, x2, y2, Id = int(x1), int(y1), int(x2), int(y2), int(Id)
        # The values of width & height
        w, h = x2 - x1, y2 - y1
        # Create a Corner Rectangle
        cvzone.cornerRect(frame, (x1, y1, w, h), l = 7, rt = 2, colorR = (255, 255, 255))
                        # l : length of the corner rectangles
                        # rt : rectangle thickness
        cvzone.putTextRect(frame, f'{Id}', (max(0, x1), max(35, y1)), scale = 1, thickness = 2, 
                           colorR = (255, 100, 50), offset = 5)
               #max(0, x1), max(35, y1)): the top-left corner of the rectangle where the text will be displayed.
                   #max() function is used to make sure that the coordinates are not negative, which can cause an error
               #scale = 1: the font scale of the text
               #thickness = 2: the thickness of the rectangle border
               #offset = 5: the distance between the text and the top border of the rectangle

        # Find the center point of a bounding box
        # (check whether the center points touch the line & if it does then only it will count)
        cx, cy = x1 + w // 2, y1 + h // 2
            #x1 + w // 2 calculates the x-coordinate of the center of the rectangle by adding the half-width 
                #to the x-coordinate of the top-left corner.
            #y1 + h // 2 calculates the y-coordinate of the center of the rectangle by adding the half-height 
                #to the y-coordinate of the top-left corner.
        cv2.circle(frame, (cx, cy), 3, (255, 255, 255), cv2.FILLED)

        # To check the limit of x & y
        # This is commonly used to determine whether an object has crossed a certain line.
        if limits[0] < cx < limits[2] and limits[1] - 20 < cy < limits[1] + 20:       
                # horizontal position & vertical position
                #limits[0] < cx < limits[2]: The x-coordinate cx is within the left and right limits
                #limits[1] - 20 < cy < limits[1] + 20: The y-coordinate cy is within a range of 20 pixels above 
                    # and below the value limits[1]
            # To count the no. of times whick Id is 1st 0 and present in the totalCounts list
            if totalCount.count(Id) == 0:
                totalCount.append(Id)      # append the value of Id
                # Show the line green color whenever it counts new detection
                cv2.line(frame, (limits[0], limits[1]), (limits[2], limits[3]), (0, 255, 0), 2)
               
    # To show the countings on graphic's image
    cv2.putText(frame, str(len(totalCount)), (138, 60), cv2.FONT_HERSHEY_PLAIN, 3, (255, 255, 255), 3)

    cv2.imshow('Car Detection', frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()